<a href="https://colab.research.google.com/github/s-mallat/AR-TypoFusion/blob/main/Sali_Mallat_MScThesis_FinalCode_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**AR+TypoFusion: A Master Thesis Project**
Sali Mallat / 20040127 <br>
MSc Creative Computing, CCI, UAL <br><br>
<hr>

### **ATTRIBUTIONS**

Stable Diffusion model belongs to Stability AI and Runway. The following code uses Stable Diffusion 1-4v that is  available on Huggingface. Kindly note that I have attached a copy of the [license](https://github.com/s-mallat/AR-TypoFusion/blob/main/LICENSES/StableDiffusion_LICENSE) for this model in my github repo. 
Credit goes to Patrick von Platen and Suraj Patil and Anton Lozhkov and Pedro Cuenca and Nathan Lambert and Kashif Rasul and Mishig Davaadorj and Thomas Wolf.

**Link to HugginFace's Stable Diffusion Github Repo:**

    https://github.com/huggingface/diffusers

Stable Diffusion is based on Latent Diffusion model by Robin Rombach, Andreas Blattmann, Dominik Lorenz, Patrick Esser, Björn Ommer.

**Link to Latent Diffusion Github Repo:**

    https://github.com/CompVis/stable-diffusion

**Published Literature on Latent Diffusion:**

    Rombach, R., Blattmann, A., Lorenz, D., Esser, P. and Ommer, B., 2022. High-resolution image synthesis with latent diffusion models. In Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition (pp. 10684-10695).


Also, please note that the Interpolation library used in this code is FILM net by Fitsum Reda and Janne Kontkanen and Eric Tabellion and Deqing Sun and Caroline Pantofaru and Brian Curless. I have also attached a copy of the [license](https://github.com/s-mallat/AR-TypoFusion/blob/main/LICENSES/FILM_LICENSE) for this repo as well.

**Link to FILM Interpolation Github Repo:**

    https://github.com/google-research/frame-interpolation

While this code has been modified and written by me, please note that I have also used other external source code which I reference when used through out this notebook.
<br><br>


<hr>

#SECTION 00<br>
#PRE-LOAD + IMPORTS

In [1]:
#Source: https://colab.research.google.com/notebooks/pro.ipynb

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Nov 21 12:29:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    43W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#Source: https://colab.research.google.com/notebooks/pro.ipynb

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.6 MB 5.0 MB/s 
     |████████████████████████████████| 54 kB 4.3 MB/s 
     |████████████████████████████████| 56 kB 6.3 MB/s 
     |████████████████████████████████| 84 kB 4.9 MB/s 
     |████████████████████████████████| 2.3 MB 77.8 MB/s 
     |████████████████████████████████| 106 kB 90.6 MB/s 
     |████████████████████████████████| 55 kB 4.9 MB/s 
     |████████████████████████████████| 84 kB 4.2 MB/s 
     |████████████████████████████████| 278 kB 68.5 MB/s 
     |████████████████████████████████| 213 kB 84.9 MB/s 
     |████████████████████████████████| 64 kB 3.5 MB/s 
     |████████████████████████████████| 80 kB 11.1 MB/s 
     |████████████████████████████████| 68 kB 8.8 MB/s 
     |████████████████████████████████| 68 kB 8.7 MB/s 
     |████████████████████████████████| 68 kB 8.9 MB/s 
     |████████████████████████████████| 46 kB 5.0 

In [4]:
#Source: https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb

!pip install diffusers==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 153 kB 5.1 MB/s 
     |████████████████████████████████| 5.5 MB 43.5 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 
     |████████████████████████████████| 182 kB 73.6 MB/s 
     |████████████████████████████████| 7.6 MB 62.0 MB/s 
     |████████████████████████████████| 1.6 MB 4.8 MB/s 


In [5]:
import shutil
from pathlib import Path
import os

In [6]:
#Source: https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb
#Source: https://www.gradio.app/docs/
#Source: https://huggingface.co/CompVis/stable-diffusion-v-1-4-original

import gradio as gr
import os
from PIL import Image, ImageDraw
import cv2
import numpy as np
from IPython.display import HTML
from base64 import b64encode

import requests
from io import BytesIO
import PIL
import sys

import torch
from torch import autocast
from torch.nn import functional as F
from tqdm.auto import tqdm
from huggingface_hub import notebook_login
from google.colab import output

from pathlib import Path

from google.colab.patches import cv2_imshow
from google.colab import files

import matplotlib as mpl
import matplotlib.pyplot as plt

import inspect
import warnings

from typing import List, Optional, Union, Tuple

from diffusers import (
    AutoencoderKL,
    DDIMScheduler,
    DiffusionPipeline,
    PNDMScheduler,
    LMSDiscreteScheduler,
    UNet2DConditionModel,
)
from diffusers.pipelines.stable_diffusion import StableDiffusionSafetyChecker

from transformers import CLIPFeatureExtractor, CLIPTextModel, CLIPTokenizer

device = 'cuda'

output.enable_custom_widget_manager()
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [7]:
#Source: https://colab.research.google.com/drive/1HuPGBn7JIAiT5PYM86Uu55ixachXsY1y#scrollTo=HBMnARHIuGiq
#Source: https://github.com/google-research/frame-interpolation

local_path = '/content/'

if not Path('./frame-interpolation').exists():
    !git clone https://github.com/pytti-tools/frame-interpolation

try:
    import frame_interpolation
except ModuleNotFoundError:
    %pip install -r ./frame-interpolation/requirements_colab.txt
    %pip install ./frame-interpolation


from loguru import logger

Cloning into 'frame-interpolation'...
remote: Enumerating objects: 179, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 179 (delta 45), reused 31 (delta 31), pack-reused 120
Receiving objects: 100% (179/179), 26.97 MiB | 69.56 MiB/s, done.
Resolving deltas: 100% (83/83), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 4.7 MB/s 
     |████████████████████████████████| 58 kB 7.3 MB/s 
     |████████████████████████████████| 506 kB 82.0 MB/s 
     |████████████████████████████████| 151 kB 72.8 MB/s 
     |████████████████████████████████| 15.3 MB 35.1 MB/s 
     |████████████████████████████████| 23.6 MB 28.7 MB/s 
     |████████████████████████████████| 62 kB 1.9 MB/s 
     |████████████████████████████████| 829 kB 88.5 MB/s 
     |████████████████████████████████| 2.4 MB 70.6 MB/s 
  Created wheel for avro-python3: fil

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./frame-interpolation
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyttitools-film: filename=pyttitools_film-0.0.1-py3-none-any.whl size=54528 sha256=71e62d805cfb1ceba25d7240f8e101591ffef09c1216ede412f6f5e4f061f90e
  Stored in directory: /root/.cache/pip/wheels/06/20/2d/805027cf9050151c8efedaa9a546edc102cadaaf13f56a6348
Successfully built pyttitool

In [8]:
#Source: https://colab.research.google.com/drive/1HuPGBn7JIAiT5PYM86Uu55ixachXsY1y#scrollTo=HBMnARHIuGiq
#Source: https://github.com/google-research/frame-interpolation

share_id = "1GhVNBPq20X7eaMsesydQ774CgGcDGkc6"

if not (Path(local_path) / 'saved_model').exists():
    !pip install --upgrade gdown
    !gdown --folder {share_id}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.5.3-py3-none-any.whl size=14840 sha256=130e29a3b95ca1d8b0e70a979f15f5e0051e5e2497ffd66843e4511787b69a17
  Stored in directory: /root/.cache/pip/wheels/94/8d/0b/bdcd83555c3555f91a33f6c2384428d9f163c7d75ab0d272b4
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
Retrieving folder list
Retrieving folder 1OmIh0a1m2yeBg0vrc9uL1aVbSvwrK6QA assets
Processing file 1-KW5CVihmeiiMki9fIqwUfsnrVJyLdyn keras_metadata.pb
Processing file 11nvcQvf5n9JMrryvIPfypYEbRYz_1egM saved_model.pb
Retrieving folder 11TJyMo3c5YR48gzMRdzbitpBlm0MQbdu variables
Processing file 1Cke9KRsEW3s50jq3RULvYbUoEY8v--zN variables.data-00000

In [9]:
#Source: https://github.com/CompVis/stable-diffusion
#Source: https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb

has_cuda = torch.cuda.is_available()
device = torch.device('cpu' if not has_cuda else 'cuda')
print(device)

cuda


In [10]:
#Source: https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb

from diffusers import StableDiffusionPipeline

os.makedirs("diffusers-cache", exist_ok=True)


pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    cache_dir="diffusers-cache",
    revision="fp16",
    torch_dtype=torch.float16,
    use_auth_token=True
).to('cuda')

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/342 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/71.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/167M [00:00<?, ?B/s]

{'trained_betas'} was not found in config. Values will be initialized to default values.


In [11]:
#Source: https://github.com/huggingface/diffusers
#Source: https://github.com/replicate/cog-stable-diffusion/blob/main/predict.py
#Source: https://github.com/huggingface/diffusers/tree/main/src/diffusers/pipelines
#Source: https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/image_2_image_using_diffusers.ipynb
#Source: https://github.com/replicate/cog-stable-diffusion

class StableDiffusionImg2ImgPipeline(DiffusionPipeline):

    def __init__(
        self,
        vae: AutoencoderKL,
        text_encoder: CLIPTextModel,
        tokenizer: CLIPTokenizer,
        unet: UNet2DConditionModel,
        scheduler: Union[DDIMScheduler, PNDMScheduler, LMSDiscreteScheduler],
    ):
        super().__init__()
        scheduler = scheduler.set_format("pt")
        self.register_modules(
            vae=vae,
            text_encoder=text_encoder,
            tokenizer=tokenizer,
            unet=unet,
            scheduler=scheduler
        )

    @torch.no_grad()
    def __call__(
        self,
        prompt: Union[str, List[str]],
        init_image: Optional[torch.FloatTensor],
        mask: Optional[torch.FloatTensor],
        width: int,
        height: int,
        prompt_strength: float = 0.8,
        num_inference_steps: int = 50,
        guidance_scale: float = 7.5,
        eta: float = 0.0,
        generator: Optional[torch.Generator] = None,
    ) -> Image:
        if isinstance(prompt, str):
            batch_size = 1
        elif isinstance(prompt, list):
            batch_size = len(prompt)
        else:
            raise ValueError(
                f"`prompt` has to be of type `str` or `list` but is {type(prompt)}"
            )

        if prompt_strength < 0 or prompt_strength > 1:
            raise ValueError(
                f"The value of prompt_strength should in [0.0, 1.0] but is {prompt_strength}"
            )

        if mask is not None and init_image is None:
            raise ValueError(
                "If mask is defined, then init_image also needs to be defined"
            )

        if width % 8 != 0 or height % 8 != 0:
            raise ValueError("Width and height must both be divisible by 8")

        # set timesteps
        accepts_offset = "offset" in set(
            inspect.signature(self.scheduler.set_timesteps).parameters.keys()
        )
        extra_set_kwargs = {}
        offset = 0
        if accepts_offset:
            offset = 1
            extra_set_kwargs["offset"] = 1

        self.scheduler.set_timesteps(num_inference_steps, **extra_set_kwargs)

        if init_image is not None:
            init_latents_orig, latents, init_timestep = self.latents_from_init_image(
                init_image,
                prompt_strength,
                offset,
                num_inference_steps,
                batch_size,
                generator,
            )
        else:
            latents = torch.randn(
                (batch_size, self.unet.in_channels, height // 8, width // 8),
                generator=generator,
                device=self.device,
            )
            init_timestep = num_inference_steps

        do_classifier_free_guidance = guidance_scale > 1.0
        text_embeddings = self.embed_text(
            prompt, do_classifier_free_guidance, batch_size
        )

        # prepare extra kwargs for the scheduler step, since not all schedulers have the same signature
        # eta (η) is only used with the DDIMScheduler, it will be ignored for other schedulers.
        # eta corresponds to η in DDIM paper: https://arxiv.org/abs/2010.02502
        # and should be between [0, 1]
        accepts_eta = "eta" in set(
            inspect.signature(self.scheduler.step).parameters.keys()
        )
        extra_step_kwargs = {}
        if accepts_eta:
            extra_step_kwargs["eta"] = eta

        mask_noise = torch.randn(latents.shape, generator=generator, device=self.device)

        # if we use LMSDiscreteScheduler, let's make sure latents are mulitplied by sigmas
        if isinstance(self.scheduler, LMSDiscreteScheduler):
            latents = latents * self.scheduler.sigmas[0]

        t_start = max(num_inference_steps - init_timestep + offset, 0)
        for i, t in tqdm(enumerate(self.scheduler.timesteps[t_start:])):
            # expand the latents if we are doing classifier free guidance
            latent_model_input = (
                torch.cat([latents] * 2) if do_classifier_free_guidance else latents
            )

            if isinstance(self.scheduler, LMSDiscreteScheduler):
                sigma = self.scheduler.sigmas[i]
                latent_model_input = latent_model_input / ((sigma ** 2 + 1) ** 0.5)

            # predict the noise residual
            noise_pred = self.unet(
                latent_model_input, t, encoder_hidden_states=text_embeddings
            )["sample"]

            # perform guidance
            if do_classifier_free_guidance:
                noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
                noise_pred = noise_pred_uncond + guidance_scale * (
                    noise_pred_text - noise_pred_uncond
                )

            # compute the previous noisy sample x_t -> x_t-1
            if isinstance(self.scheduler, LMSDiscreteScheduler):
                latents = self.scheduler.step(noise_pred, i, latents, **extra_step_kwargs)[
                    "prev_sample"
                ]
            else:
                latents = self.scheduler.step(noise_pred, t, latents, **extra_step_kwargs)[
                    "prev_sample"
                ]

            # replace the unmasked part with original latents, with added noise
            if mask is not None:
                timesteps = self.scheduler.timesteps[t_start + i]
                timesteps = torch.tensor(
                    [timesteps] * batch_size, dtype=torch.long, device=self.device
                )
                noisy_init_latents = self.scheduler.add_noise(init_latents_orig, mask_noise, timesteps)
                latents = noisy_init_latents * mask + latents * (1 - mask)

        # scale and decode the image latents with vae
        latents = 1 / 0.18215 * latents
        image = self.vae.decode(latents)['sample']

        image = (image / 2 + 0.5).clamp(0, 1)
        image = image.cpu().permute(0, 2, 3, 1).numpy()

        image = self.numpy_to_pil(image)

        return {"sample": image}


    def latents_from_init_image(
        self,
        init_image: torch.FloatTensor,
        prompt_strength: float,
        offset: int,
        num_inference_steps: int,
        batch_size: int,
        generator: Optional[torch.Generator],
    ) -> Tuple[torch.FloatTensor, torch.FloatTensor, int]:
        # encode the init image into latents and scale the latents
        init_latents = self.vae.encode(init_image.to(self.device)).latent_dist.sample()
        init_latents = 0.18215 * init_latents
        init_latents_orig = init_latents

        # prepare init_latents noise to latents
        init_latents = torch.cat([init_latents] * batch_size)

        # get the original timestep using init_timestep
        init_timestep = int(num_inference_steps * prompt_strength) + offset
        init_timestep = min(init_timestep, num_inference_steps)
        timesteps = self.scheduler.timesteps[-init_timestep]
        timesteps = torch.tensor(
            [timesteps] * batch_size, dtype=torch.long, device=self.device
        )

        # add noise to latents using the timesteps
        noise = torch.randn(init_latents.shape, generator=generator, device=self.device)
        init_latents = self.scheduler.add_noise(init_latents, noise, timesteps)

        return init_latents_orig, init_latents, init_timestep

    def embed_text(
        self,
        prompt: Union[str, List[str]],
        do_classifier_free_guidance: bool,
        batch_size: int,
    ) -> torch.FloatTensor:
        # get prompt text embeddings
        text_input = self.tokenizer(
            prompt,
            padding="max_length",
            max_length=self.tokenizer.model_max_length,
            truncation=True,
            return_tensors="pt",
        )
        text_embeddings = self.text_encoder(text_input.input_ids.to(self.device))[0]

        # here `guidance_scale` is defined analog to the guidance weight `w` of equation (2)
        # of the Imagen paper: https://arxiv.org/pdf/2205.11487.pdf . `guidance_scale = 1`
        # corresponds to doing no classifier free guidance.
        # get unconditional embeddings for classifier free guidance
        if do_classifier_free_guidance:
            max_length = text_input.input_ids.shape[-1]
            uncond_input = self.tokenizer(
                [""] * batch_size,
                padding="max_length",
                max_length=max_length,
                return_tensors="pt",
            )
            uncond_embeddings = self.text_encoder(
                uncond_input.input_ids.to(self.device)
            )[0]

            # For classifier free guidance, we need to do two forward passes.
            # Here we concatenate the unconditional and text embeddings into a single batch
            # to avoid doing two forward passes
            text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

        return text_embeddings

In [12]:
#Source: https://github.com/replicate/cog-stable-diffusion

scheduler = PNDMScheduler(
            beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear"
        )

In [13]:
#Source: https://github.com/replicate/cog-stable-diffusion

MODEL_CACHE = "diffusers-cache"
pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(
            "CompVis/stable-diffusion-v1-4",
            scheduler=scheduler,
            revision="fp16",
            torch_dtype=torch.float16,
            cache_dir=MODEL_CACHE,
            local_files_only=True,
        ).to("cuda")

#SECTION 01 <br>
#GENERATE STILL IMAGE

In [14]:
#Source: https://pyimagesearch.com/2016/10/31/detecting-multiple-bright-spots-in-an-image-with-python-and-opencv/
#Source: https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html
#Source: https://github.com/tiangolo/fastapi/discussions/4308
#Source: https://techtutorialsx.com/2018/06/24/python-opencv-saving-an-image-to-the-file-system/

def preprocess_upload(image: Image):
    dim = (512, 512)
    image = Image.open(image)
    image.save('/content/uploaded.png','PNG')
    image = "/content/uploaded.png"
    image = cv2.imread(image)
    image = np.invert(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY)[1]
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    image = cv2.imwrite('preprocessed.png', image)
    return image

In [15]:
#Source: https://docs.opencv.org/3.4/de/d25/imgproc_color_conversions.html
#Source: https://docs.opencv.org/4.x/da/d22/tutorial_py_canny.html

def mask_outline(image):
  edges = cv2.Canny(image,512,512, L2gradient = True)
  kernel = np.ones((5,5),np.uint8)
  mask3 = cv2.dilate(edges, kernel, iterations=1)
  mask3 = np.invert(mask3)
  mask3 = cv2.cvtColor(mask3, cv2.COLOR_BGR2RGB)
  return mask3

In [16]:
#Source: https://docs.opencv.org/4.x/d0/d86/tutorial_py_image_arithmetics.html

def reconstruct_Img(image):
  Gen1 = cv2.imread('/content/baseGen1.png')
  Gen2 = cv2.imread('/content/baseGen2.png')

  image = cv2.imread('/content/preprocessed.png', cv2.IMREAD_COLOR)
  mask1 = image
  mask2 = np.invert(mask1)
  mask3 = mask_outline(mask1)

  cv2.imwrite('/content/mask1.jpg', mask1)
  cv2.imwrite('/content/mask2.jpg', mask2)
  cv2.imwrite('/content/mask3.jpg', mask3)

  #GENERATED IMAGE 1 (HALF)
  img1_1 = cv2.bitwise_and(mask1, Gen1)
  img1_2 = cv2.bitwise_and(mask2, Gen2)

  img1_1_in = np.invert(img1_1)
  img1_2_in = np.invert(img1_2)

  #GENERATED IMAGE 2 (HALF)
  img2_1 = cv2.bitwise_and(mask1, Gen2)
  img2_2 = cv2.bitwise_and(mask2, Gen1)

  img2_1_in = np.invert(img2_1)
  img2_2_in = np.invert(img2_2)

  #FULL GENERATED IMAGE 1
  imgFull_1 = cv2.bitwise_and(img1_1_in, img1_2_in)
  imgFull_1 = np.invert(imgFull_1)

  #FULL GENERATED IMAGE 2
  imgFull_2 = cv2.bitwise_and(img2_1_in, img2_2_in)
  imgFull_2 = np.invert(imgFull_2)

  FinalImg1 = cv2.bitwise_and(mask3, imgFull_1)
  FinalImg2 = cv2.bitwise_and(mask3, imgFull_2)

  cv2.imwrite('final_02.jpg', FinalImg2)

  image = cv2.imwrite('final_01.jpg', FinalImg1)
  image = cv2.imread('final_01.jpg')

  return FinalImg1

In [17]:
#Source: https://docs.opencv.org/4.x/d0/d86/tutorial_py_image_arithmetics.html

def reconstruct_Img2(image):
  Gen1 = cv2.imread('/content/baseGen1.png')
  Gen2 = cv2.imread('/content/baseGen2.png')

  image = cv2.imread('/content/preprocessed.png', cv2.IMREAD_COLOR)
  mask1 = image
  mask2 = np.invert(mask1)
  mask3 = mask_outline(mask1)

  cv2.imwrite('/content/mask1.jpg', mask1)
  cv2.imwrite('/content/mask2.jpg', mask2)
  cv2.imwrite('/content/mask3.jpg', mask3)

  #GENERATED IMAGE 1 (HALF)
  img1_1 = cv2.bitwise_and(mask1, Gen1)
  img1_2 = cv2.bitwise_and(mask2, Gen2)

  img1_1_in = np.invert(img1_1)
  img1_2_in = np.invert(img1_2)

  #GENERATED IMAGE 2 (HALF)
  img2_1 = cv2.bitwise_and(mask1, Gen2)
  img2_2 = cv2.bitwise_and(mask2, Gen1)

  img2_1_in = np.invert(img2_1)
  img2_2_in = np.invert(img2_2)

  #FULL GENERATED IMAGE 1
  imgFull_1 = cv2.bitwise_and(img1_1_in, img1_2_in)
  imgFull_1 = np.invert(imgFull_1)

  #FULL GENERATED IMAGE 2
  imgFull_2 = cv2.bitwise_and(img2_1_in, img2_2_in)
  imgFull_2 = np.invert(imgFull_2)

  FinalImg1 = cv2.bitwise_and(mask3, imgFull_1)
  FinalImg2 = cv2.bitwise_and(mask3, imgFull_2)

  cv2.imwrite('final_02.jpg', FinalImg2)

  #image = cv2.imwrite('final_01.jpg', FinalImg1)
  #image = cv2.imread('final_01.jpg')
  image = cv2.imwrite('final_noOutline.jpg', imgFull_1)
  image = cv2.imread('final_noOutline.jpg')

  return FinalImg2

In [18]:
#Source: https://gradio.app/docs/

def infer(init_image, prompt1, prompt2):
  
  image = preprocess_upload(init_image)

  with torch.autocast('cuda'):
    Gen1 = pipe(prompt1, guidance_scale=7.5, num_inference_steps=50)["sample"][0]
    Gen2 = pipe(prompt2, guidance_scale=7.5, num_inference_steps=50)["sample"][0]
  
  Gen1.save("baseGen1.png")
  Gen2.save("baseGen2.png")

  output = reconstruct_Img(image)
  image1RGB = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)

  output2 = reconstruct_Img2(image)
  image2RGB = cv2.cvtColor(output2, cv2.COLOR_BGR2RGB)

  return image1RGB

In [ ]:
# TESTING

# init_image = "/content/Dream-01.jpg"
# prompt1 = "a watercolor painting of white snow colors"
# prompt2 = "a watercolor deep blue texture"
# infer(init_image,prompt1,prompt2)

#SECTION 02 <br>
#GENERATE VIDEO

In [19]:
#Source: https://github.com/replicate/cog-stable-diffusion/blob/main/predict.py
#Source: https://github.com/huggingface/diffusers/blob/main/src/diffusers/pipelines/stable_diffusion/pipeline_onnx_stable_diffusion_img2img.py

def preprocess_init_image(image: Image, width: int, height: int):
    image = image.resize((width, height), resample=Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.0 * image - 1.0

In [20]:
#Source: https://github.com/replicate/cog-stable-diffusion/blob/main/predict.py
#Source: https://github.com/huggingface/diffusers/blob/main/src/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion_inpaint.py
#Source: https://github.com/huggingface/diffusers/blob/main/src/diffusers/pipelines/stable_diffusion/pipeline_onnx_stable_diffusion_img2img.py

def preprocess_mask(mask: Image, width: int, height: int):
    mask = mask.convert("L")
    mask = mask.resize((width // 8, height // 8), resample=Image.LANCZOS)
    mask = np.array(mask).astype(np.float32) / 255.0
    mask = np.tile(mask, (4, 1, 1))
    mask = mask[None].transpose(0, 1, 2, 3)  # what does this step do?
    mask = torch.from_numpy(mask)
    return mask

In [21]:
#Source: https://techtutorialsx.com/2018/06/24/python-opencv-saving-an-image-to-the-file-system/
#Source: https://www.geeksforgeeks.org/create-a-directory-in-python/

def generate_frames(init_image, prompt1, prompt2):
  init_path = Image.open("/content/final_01.jpg").convert("RGB")
  mask_path = Image.open('/content/mask1.jpg')
  mask_path_2 = Image.open('/content/mask2.jpg')

  init_image = preprocess_init_image(image=init_path, width=512, height=512).to("cuda")
  mask = preprocess_mask(mask=mask_path, width=512, height=512).to("cuda")
  mask_Invert = preprocess_mask(mask=mask_path_2, width=512, height=512).to("cuda")

  generator = torch.Generator("cuda").manual_seed(-1)

  with torch.autocast("cuda"):
    Frames_mask = pipeline(prompt=[prompt1] * 3, init_image=init_image, mask=mask, width=512, height=512, prompt_strength=0.75, num_inference_steps=100, guidance_scale=7.5, generator=generator)
    Frames_maskInvert = pipeline(prompt=[prompt2] * 3, init_image=init_image, mask=mask_Invert, width=512, height=512, prompt_strength=0.75, num_inference_steps=100, guidance_scale=7.5, generator=generator)

  # CREATE A NEW FOLDER FOR ANIMATION FRAMES
  if os.path.exists('/content/frames'):
    shutil.rmtree('/content/frames')
    frames = os.mkdir('/content/frames')
  else:
    frames = os.mkdir('/content/frames')

  # FRAMES
  init_path.save("/content/frames/Frames_00.jpg")
  Frames_mask['sample'][0].save("/content/frames/Frames_01.jpg")
  Frames_maskInvert['sample'][0].save("/content/frames/Frames_02.jpg")
  Frames_mask['sample'][1].save("/content/frames/Frames_03.jpg")
  Frames_maskInvert['sample'][1].save("/content/frames/Frames_04.jpg")
  Frames_mask['sample'][2].save("/content/frames/Frames_05.jpg")
  Frames_maskInvert['sample'][2].save("/content/frames/Frames_06.jpg")
  init_path.save("/content/frames/Frames_07.jpg")

  return frames

In [ ]:
# TESTING

# init_image = "/content/Dream-01.jpg"
# prompt1 = "a dream-likewatercolor painting of white snow colors"
# prompt2 = "galaxies colliding in watercolor"
# generate_frames(init_image,prompt1,prompt2)

In [22]:
#Source: https://gradio.app/docs/
#Source: https://colab.research.google.com/drive/1HuPGBn7JIAiT5PYM86Uu55ixachXsY1y#scrollTo=HBMnARHIuGiq

def video(init_image, prompt1, prompt2):
  frames_dir = "/content/frames"
  recursive_interpolation_passes = 1
  output_video = True
  output_video_fps = 40

  generate_frames(init_image,prompt1,prompt2)

  logger.info("Beginning interpolation...")

  if output_video:
    !python -m frame_interpolation.eval.interpolator_cli \
        --model_path ./saved_model \
        --pattern {frames_dir} \
        --fps {output_video_fps} \
        --output_video
        
  else:
      !python -m frame_interpolation.eval.interpolator_cli \
        --model_path ./saved_model \
        --pattern {frames_dir} \

  logger.info("Interpolation comlpete.")
  
  output = "/content/frames/interpolated.mp4"
  return output

In [ ]:
# TESTING

# init_image = "/content/uploaded.png"
# prompt1 = "painting of magical lake in japan | oil painting"
# prompt2 = "painting of magical forest in japan | oil painting"
# output_video_fps = 40

# video(init_image, prompt1, prompt2, output_video_fps)

#SECTION 03 <br>
#WEB APP

In [23]:
#Source: https://gradio.app/docs/
#Source: https://huggingface.co/spaces/stabilityai/stable-diffusion/blob/main/app.py

css = """
        .gradio-container {
            font-family: 'IBM Plex Sans', sans-serif;
            background-color: #ceb4ff;
            border: none;
            font-size: 25px;
        }
        
        .gr-button {
            color: #152be0;
            background: #cbf952;
            border-color: #152be0;
            border-width:2px;
            border-radius: 0%;
            padding: 25px;
        }
        
        .gr-button-lg {
            border-radius: 2px;
            padding: .5rem 1rem;
            font-size: 25px;
            line-height: 1.5rem;
            font-weight: 400;
        }
        
        .bg-white {
            color: #cbf952;
            background: #152be0;
            border-color: #152be0;  
            border-radius: 0%;
            border-width:2px;
            border-size:40px;
        }
        .container {
            max-width: 1000px;
            margin: auto;
            padding-top: 20px;
            border-radius: 0%;
            border: none;
            background-color: #ceb4ff;
        }
        
        #title {
          color: #152be0;
          font-size: 70px;
          font-style: bold;
          line-height: 1.0;
          text-align: center;
        }
        
        #prompt-text-input{
          background-color: #ceb4ff;
          border-color: #d4ff5d;
          border-radius: 0%;
          text-align: center;
          font-size: 20px;
        }
        
        #gallery {
            min-height: 512px;
            margin-bottom: 15px;
            margin-left: auto;
            margin-right: auto;
            border-bottom-right-radius: .5rem !important;
            border-bottom-left-radius: .5rem !important;
        }
        
        
        .gr-button:hover{
            color: #cbf952;
            background: #152be0;
            border-color: #152be0;    
        }
        
        .bg-white:hover{
            color: #ceb4ff;
            background-color: #152be0;
            border-color: #152be0;    
        }

    
        #prompt-container{
            gap: 0;
            border-color: #ceb4ff;
            border-radius: 0%;
            background-color: #ceb4ff;
            text-align: center;
            font-size: 20px;
            
        }
        
        #image-box{
            min-height: 512px;
        }
        
        
        .gr-form {
            border-color: #ceb4ff;
            border-width: 2px;
            border-radius: 0%;
        }
        
        // Box around input & output div
        .gr-block {
            border-color: #152be0;
            border-width: 2px;
            border-radius: 0%;
        }
        
        .gr-box {
            position: relative;
            border-radius: 0%;
            border-color: #152be0;
            border-width: 2px;
            border-style: solid;
            background-color:#ceb4ff ;
            font-size: 24px;
            line-height: 1.25rem;
            color: #152be0;
            text-align: center;
        }
        
        
        input, textarea {
            border-radius: 0%;
            font-family: inheret;
            background: #ceb4ff;
        }
        
        .gr-text-input {
            padding: .625rem;
            box-shadow: none;
            border-radius: 0%;
            border-color: #152be0;
            border-width: 2px;
            }
            
            
        .gr-input {
            background-color: #ceb4ff;
            color: #152be0;
            }
            
            *, ::before, ::after {
    box-sizing: border-box;
    border-width: 0;
    /* border-style: solid; */
    border-color: #e5e7eb;
}

.text-gray-500 {
    opacity: 0;
    color: #e5e7eb;
}

.gr-padded {
padding: 0px;
}

*, :before, :after {
    box-sizing: border-box;
    border-width: 0;
    border-style: solid;
    border-color:#e5e7eb}
    
.input::placeholder, textarea::placeholder {
    color: black;
    font-size: 25px;
    opacity:1;
    text-align:center;
    
}

.p-2{
padding:0px;
}

.border-2 {
    border: 0px;
    border-radius: 0%;
}

.border-b-2 {
    border: 0px;
    border-radius: 0%;
}

.gr-form > .gr-block {
    border-radius: 0;
    border-width: 0px;
}

.h-60 {
    height: 512px;
}
 
.text-gray-400 {
    color: #152be0;
    font-size: 25px;
}

.items-center {
    align-items: center;
}

"""

In [24]:
#Source: https://gradio.app/docs/
#Source: https://huggingface.co/spaces/stabilityai/stable-diffusion/blob/main/app.py

with gr.Blocks(css=css) as demo:
    gr.Markdown("AR+TypoFusion - App Demo", elem_id="title")

############## SECTION 01 ##############
    #INPUTS
    prompt1_input = gr.Textbox(elem_id="prompt-text-input", placeholder="Enter Prompt #1").style(
                    container=True,
                )
    prompt2_input = gr.Textbox(elem_id="prompt-text-input", placeholder="Enter Prompt #2").style(
                    container=True,
                )

    with gr.Tab("Image"):

      with gr.Row(elem_id="prompt-container"):
        with gr.Column(scale=1):
            image_upload = gr.Image(
                label = "Upload Image",
                type ="filepath",
                elem_id="image-box"
                )

        #OUTPUT
        with gr.Column(scale=1):
            image_output = gr.Image(type="pil", image_mode="BGR", elem_id="gallery")
      generate_button = gr.Button("Generate Image").style(
                    full_width=True,
                    container=True,
                )
        
#             flip_button = gr.Button("Flip", elem_id="small-button").style(
#                     full_width=True,
#                     container=True,
#                 )
#             download_button = gr.Button("Download", elem_id="small-button").style(
#                     full_width=True,
#                     container=True,
#                 )




############## SECTION 02 ##############
    with gr.Tab("Video"):
        #INPUTS
#         output_video_fps = gr.Slider(label="Frame Rate", minimum=10, maximum=60, step=1)
        #OUTPUT
        video_output = gr.Video(elem_id="gallery")

        generateVid_button = gr.Button("Generate Video").style(
                    full_width=False,
                )
    

############## RUN ##############
    generate_button.click(infer, inputs=[image_upload,prompt1_input,prompt2_input], outputs=image_output)
    generateVid_button.click(video, inputs=[image_upload, prompt1_input, prompt2_input], outputs=video_output)


    print("Everything is working fine !")
demo.launch()

Everything is working fine !
Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [28]:
#Source: https://pynative.com/python-delete-files-and-directories/

shutil.rmtree('/content/frames/')